```

Sip = [S_x; S_y; S_z];
TCP = [X; Y; Z];

Rx = [1, 0, 0; 0, cos(alpha), sin(alpha); 0, -sin(alpha), cos(alpha)];
Ry = [cos(beta), 0, -sin(beta); 0, 1, 0; sin(beta), 0, cos(beta)];
Rz = [cos(gama), sin(gama), 0; -sin(gama), cos(gama), 0; 0, 0, 1];
R = Rx*Ry*Rz;

v = [v1; v2; v3];
x = [X; Y; Z]-L*R*v;
o = R*v;


a = sqrt(((-R(1,:)*v*TCP(3))/(R(3,:)*v))^2+(-(R(2,:)*v)*TCP(3)/(R(3,:)*v))^2+(-TCP(3))^2);
Q = a*o+TCP;
Sip_proj = Sip+(o'*Q-o'*Sip)*(x-Sip)/(o'*x-o'*Sip);
s_2 = [n1; n2; n3];
s_1 = cross(o,s_2);
k_1 = ((Sip_proj-Q)*s_1)/norm(s_1)^2;
k_2 = ((Sip_proj-Q)*s_2)/norm(s_2)^2;
Q_t = [0; 0];

% Obraz v zobrazovaci rovine
f_1 = (Q_t(1)+k_1*ff/(a+L-ff));
f_2 = Q_t(1)+k_2*ff/(a+L-ff);

% Derivace:
df1_X = diff(f_1, X);
df1_Y = diff(f_1, Y);
df1_Z = diff(f_1, Z);
df1_alpha = diff(f_1, alpha);
df1_beta = diff(f_1, beta);
df1_gama = diff(f_1, gama);
 
df2_X = diff(f_2, X);
df2_Y = diff(f_2, Y);
df2_Z = diff(f_2, Z);
df2_alpha = diff(f_2, alpha);
df2_beta = diff(f_2, beta);
df2_gama = diff(f_2, gama);

```

In [ ]:
import sympy as sp

# Define the symbols
alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S_x, S_y, S_z = sp.symbols(
    'alpha beta gama X Y Z L v1 v2 v3 n1 n2 n3 ff S_x S_y S_z', real=True)

# Sip and TCP
Sip = sp.Matrix([S_x, S_y, S_z])
TCP = sp.Matrix([X, Y, Z])

# Rotation matrices Rx, Ry, Rz
Rx = sp.Matrix([[1, 0, 0], [0, sp.cos(alpha), -sp.sin(alpha)], [0, sp.sin(alpha), sp.cos(alpha)]])
Ry = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta)], [0, 1, 0], [-sp.sin(beta), 0, sp.cos(beta)]])
Rz = sp.Matrix([[sp.cos(gama), -sp.sin(gama), 0], [sp.sin(gama), sp.cos(gama), 0], [0, 0, 1]])
R = sp.simplify(Rx * Ry * Rz)

# Vector v and calculations for x and o
v = sp.Matrix([v1, v2, v3])
x = sp.Matrix([X, Y, Z]) - L * R * v
o = R * v

# Calculate 'a' using dot products
a = sp.simplify(sp.sqrt(((-R.row(0).dot(v) * Z) / (R.row(2).dot(v)))**2 +
                        (-(R.row(1).dot(v) * Z) / (R.row(2).dot(v)))**2 + Z * Z))

# Calculate Q
Q = a * o + TCP

# Project Sip onto the line defined by o and Q
Sip_proj = Sip + (o.dot(Q) - o.dot(Sip)) * (x - Sip) / (o.dot(x) - o.dot(Sip))

# s_2 vector and s_1 as the cross product of o and s_2
s_2 = sp.Matrix([n1, n2, n3])
s_1 = o.cross(s_2)

# Calculate k_1 and k_2
k_1 = (Sip_proj - Q).dot(s_1) / s_1.norm()**2
k_2 = (Sip_proj - Q).dot(s_2) / s_2.norm()**2

# Q_t as a zero vector (for 2D image plane coordinates)
Q_t = sp.Matrix([0, 0])

# Calculate image plane coordinates f_1 and f_2
f_1 = Q_t[0] + k_1 * ff / (a + L - ff)
f_2 = Q_t[1] + k_2 * ff / (a + L - ff)
f_both = sp.Matrix([f_1, f_2])

In [ ]:


# Define the vector of symbols
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

# Compute the gradient of f_1 with respect to the vector v
gradient_f1 = sp.derive_by_array(f_1, v)


In [ ]:
gradient_f1

In [ ]:
reduced_exprs, simplified_expr = sp.cse(gradient_f1)
subs_dict = {}
for var, subexpr in reduced_exprs:
    if subexpr.func == sp.Symbol:
        subs_dict[var] = subexpr
    else:
        subs_dict[var] = subexpr
subs_dict

In [ ]:

import sympy as sp


tmp = reduced_exprs[35][1]

octave_code = sp.printing.octave_code(simplified_expr)
print(octave_code)


In [ ]:
import inspect
gradient_f1_lambd = sp.lambdify(v, gradient_f1, modules='sympy',cse=True)
print(inspect.getsource(gradient_f1_lambd))

In [ ]:
subs_dict = {}
for var, subexpr in reduced_exprs:
    if subexpr.func == sp.Symbol:
        subs_dict[var] = subexpr
    else:
        subs_dict[var] = subexpr
subs_dict

In [ ]:
reduced_exprs, simplified_expr = sp.cse(gradient_f1[3])
reduced_exprs[140][1].subs(subs_dict).subs(subs_dict).subs(subs_dict).subs(
    subs_dict).subs(subs_dict).subs(subs_dict).subs(subs_dict)

In [ ]:
import numpy as np
import inspect
f1_lambd = sp.lambdify(v, f_1, modules=["mpmath"],cse=True)

print(inspect.getsource(f1_lambd))


In [ ]:
df1_X = sp.diff(f_1, X)
df1_Y = sp.diff(f_1, Y)
df1_Z = sp.diff(f_1, Z)
df1_alpha = sp.diff(f_1, alpha)
df1_beta = sp.diff(f_1, beta)
df1_gama = sp.diff(f_1, gama)

df2_X = sp.diff(f_2, X)
df2_Y = sp.diff(f_2, Y)
df2_Z = sp.diff(f_2, Z)
df2_alpha = sp.diff(f_2, alpha)
df2_beta = sp.diff(f_2, beta)
df2_gama = sp.diff(f_2, gama)

In [ ]:
import sympy as sp

# Define the symbols
S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15 = sp.symbols("S1:16", real=True)

# Create the matrix
M = sp.Matrix(5, 3, (S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15))

# Print the matrix
print(M)

In [ ]:
# Define the matrix to store the results
results = sp.Matrix(10, 1, [0] * 10)

# Define the loop
for i in range(5):
    # Substitute the symbols from M for S_x, S_y, S_z in f_1 and f_2
    f1_i = f_1.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    f2_i = f_2.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    
    # Store the results in the matrix
    results[2 * i, 0] = f1_i * 1e3
    results[2 * i + 1, 0] = f2_i * 1e3
    


In [ ]:
import sympy as sp

f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10 = sp.symbols('f_obs1:11', real=True)

# Define the symbols
f_obs = sp.Matrix(10, 1, (f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10))

# Print the vector
print(f_obs)

In [ ]:
diff = results - f_obs
F_obs = diff.norm()**2

In [ ]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, 
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
F_obs_lambd = sp.lambdify(args, F_obs, modules='numpy', cse=True)

print(inspect.getsource(F_obs_lambd))

In [ ]:
# Define the vector of symbols
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

# Compute the gradient of f_1 with respect to the vector v
gradient_F_obs = sp.derive_by_array(F_obs, v)

In [ ]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff,
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
gradient_F_obs_lambd = sp.lambdify(args, gradient_F_obs, modules='numpy', cse=True)

print(inspect.getsource(gradient_F_obs_lambd))

In [ ]:
import sympy as sp
# Define the vector of symbols
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

# Compute the gradient of f_1 with respect to the vector v

# Create a 6x6 matrix in sympy
hessian_F_obs = sp.Matrix(6, 6, [0] * 36)

for i in range(6):
    for j in range(6):
        print(f"Done with {i}, {j}")
        hessian_F_obs[i, j] = sp.diff(gradient_F_obs[i], v[j])

In [ ]:
print(hessian_F_obs[0,0])

In [ ]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff,
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
hessian_F_obs_lambd = sp.lambdify(args, hessian_F_obs[3, 0], modules='numpy', cse=True)

print(inspect.getsource(hessian_F_obs_lambd))

In [ ]:
import numpy as np

res_np = np.zeros((6,6))
for i in range(6):
    for j in range(6):
        xpr_term, xpr_res = sp.cse(hessian_F_obs[i, j])
        res_np[i, j] = len(xpr_term)

res_np

In [ ]:
list(range(6,6))

In [ ]:
for i in range(6):
    for j in range(i+1,6):
        hessian_F_obs[i, j] = 0

In [ ]:
xpr_term, xpr_res = sp.cse(hessian_F_obs)

In [ ]:
len(xpr_term)

In [ ]:
subs_dict = {}
for var, subexpr in xpr_term:
    subs_dict[var] = subexpr
subs_dict

In [ ]:
subs_dict1 = {}
for var, subexpr in xpr_term:
    if str(subexpr).find('Derivative') != -1:
        subs_dict1[var] = subexpr.subs(subs_dict)
subs_dict1

In [ ]:
zero_dict = {key: 0 for key in subs_dict1.keys()}
zero_dict

In [ ]:
# Drop subs_dict entries from subs_dict1.keys()
for key in zero_dict:
    if key in subs_dict:
        subs_dict.pop(key)

# Substitute each entry in subs_dict for 0 in subs_dict1
for key in subs_dict:
    subs_dict[key] = subs_dict[key].subs(zero_dict)


In [ ]:
for key in subs_dict:
    if subs_dict[key] == 0:
        print(key)
        zero_dict[key] = 0


In [ ]:
subs_dict2 = {}
for var, subexpr in xpr_term:
    if str(subexpr).find('**') != -1:
        if str(subexpr.subs(subs_dict))[0:4] == 'sign':
            subs_dict2[var] = subexpr.subs(subs_dict)
subs_dict2

In [ ]:
one_dict = {key: 1 for key in subs_dict2.keys()}
one_dict

In [ ]:
# Drop subs_dict entries from subs_dict1.keys()
for key in one_dict:
    if key in subs_dict:
        subs_dict.pop(key)

# Substitute each entry in subs_dict for 0 in subs_dict1
for key in subs_dict:
    subs_dict[key] = subs_dict[key].subs(one_dict)

In [ ]:
for key in subs_dict:
    if subs_dict[key] == 1:
        print(key)
        one_dict[key] = 1

In [ ]:
with open('tmp.txt', 'w') as f:
    for key in subs_dict:
        f.write(str(key) + " = " + str(subs_dict[key]) + ";\n")

In [ ]:
xpr_res[0][1,1][0]

In [ ]:

subs_res = list()
for i in range(6):
    tmp = list()
    for j in range(6):
        try:
            tmp.append(xpr_res[0][i,j][0].subs(zero_dict).subs(one_dict))
        except:
            tmp.append(0)
    subs_res.append(tmp)


In [ ]:
with open('tmp.txt', 'a') as f:
    for i in range(6):
        for j in range(6):
            f.write("res(" + str(i+1) + "," + str(j+1) + ") = " + str(subs_res[i][j]) + ";\n")